In [ ]:
# Enable GPU: Runtime -> Change runtime type -> GPU
# Install Libraries
!pip install transformers datasets accelerate

In [ ]:
from datasets import load_dataset
from transformers import GPT2Tokenizer

# --- Setup ---
dataset_name = "DialogueCharacter/english_preference_hh_helpful_unfiltered"
dataset = load_dataset(dataset_name)

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# GPT-2 requires the EOS token for padding and a separator
tokenizer.pad_token = tokenizer.eos_token 
SEP_TOKEN = tokenizer.eos_token 


In [ ]:

# --- Formatting Function ---

def format_and_tokenize_chosen(examples):
    formatted_texts = []
    
    # We iterate over the 'instruction' and 'chosen' columns
    for instruction, chosen_response in zip(examples['instruction'], examples['chosen']):
        
        # We format the data to teach the model to generate the helpful response 
        # given the instruction. This is the Supervised Fine-Tuning step.
        text = f"User: {instruction}\nModel: {chosen_response}{SEP_TOKEN}"
        formatted_texts.append(text)

    return tokenizer(
        formatted_texts, 
        truncation=True, 
        max_length=512, # Standard max length
        padding='max_length' 
    )


In [ ]:

# --- Apply Mapping ---

# We only use the 'train' split for training
tokenized_datasets = dataset['train'].map(
    format_and_tokenize_chosen, 
    batched=True, 
    # Remove all the original columns, as we only need the tokenized IDs and labels
    remove_columns=['instruction', 'chosen', 'rejected'] 
    # NOTE: The actual dataset may have more columns. If you get a KeyError here, 
    # try running: print(dataset['train'].column_names) 
    # and adjust the list above to match.
)

# Set the 'input_ids' as 'labels' for Causal Language Modeling
tokenized_datasets = tokenized_datasets.map(
    lambda examples: {'labels': examples['input_ids']}, 
    batched=True
)

print("--- Tokenized Dataset Structure ---")
print(tokenized_datasets)

In [ ]:
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer, DataCollatorForLanguageModeling

# --- Load Model ---
model = GPT2LMHeadModel.from_pretrained(model_name)

# --- Define Training Arguments ---
output_dir = "./gpt2_helpful_sft"

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=3,                   
    per_device_train_batch_size=4,        # Adjust based on GPU memory (Colab standard is often 4-8)
    save_steps=5000,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=500,
    learning_rate=5e-5,                   
    fp16=True,                            # Use mixed precision for efficiency
)

# --- Define Data Collator ---
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False # Causal Language Modeling
)

# --- Initialize and Train ---
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets, # Pass the tokenized 'train' split
)


In [ ]:

print("Starting training...")
trainer.train()


In [ ]:

# --- Save Model ---
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model saved to {output_dir}")